In [1]:
import pandas as pd

In [2]:
train=pd.read_csv("train_wn75k28.csv")
test=pd.read_csv("test_Wf7sxXF.csv")

In [3]:
## products_purchased , signup_date

In [4]:
train["created_at"]=pd.to_datetime(train["created_at"])
train["signup_date"]=pd.to_datetime(train["signup_date"])

In [5]:
test["created_at"]=pd.to_datetime(test["created_at"])
test["signup_date"]=pd.to_datetime(test["signup_date"])

In [6]:
train['signup_date']=train['signup_date'].ffill()
train["products_purchased"]=train["products_purchased"].fillna(0)
train["created_at_year"]=pd.to_datetime(train["created_at"]).dt.year
train["created_at_month"]=pd.to_datetime(train["created_at"]).dt.month
train["created_at_day"]=pd.to_datetime(train["created_at"]).dt.day
train["signup_date_year"]=pd.to_datetime(train["signup_date"]).dt.year
train["signup_date_month"]=pd.to_datetime(train["signup_date"]).dt.month
train["signup_date_day"]=pd.to_datetime(train["signup_date"]).dt.day
train=train.drop(["created_at","signup_date"],axis=1)

In [7]:
test["products_purchased"]=test["products_purchased"].fillna(0)
test['signup_date']=test['signup_date'].ffill()
test["created_at_year"]=pd.to_datetime(test["created_at"]).dt.year
test["created_at_month"]=pd.to_datetime(test["created_at"]).dt.month
test["created_at_day"]=pd.to_datetime(test["created_at"]).dt.day
test["signup_date_year"]=pd.to_datetime(test["signup_date"]).dt.year
test["signup_date_month"]=pd.to_datetime(test["signup_date"]).dt.month
test["signup_date_day"]=pd.to_datetime(test["signup_date"]).dt.day
test=test.drop(["created_at","signup_date"],axis=1)

In [8]:
X=train.drop(["buy","id"],axis=1)
y=train["buy"]

In [9]:
X_test1=test.drop(["id"],axis=1)

## Standard Scalar

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
scaler = StandardScaler()
X=scaler.fit_transform(X)
X_test1=scaler.fit_transform(X_test1)

## Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
lr_clf=LogisticRegression(n_jobs=-1).fit(X,y)
lr_clf.score(X,y)
predicted_values=lr_clf.predict(X_test1)

In [14]:
results=pd.DataFrame({"id":test["id"],"buy":predicted_values})
results.to_csv("results.csv",index=False)

## LGBM

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
lgbm_clf=LGBMRegressor()
lgbm_clf.fit(X,y)
lgbm_clf.score(X,y)

In [ ]:
predicted_probability=lgbm_clf.predict(X_test1)
predicted_flag=[1 if x>0.5 else 0 for x in predicted_probability]

In [ ]:
results2=pd.DataFrame({"id":test["id"],"buy":predicted_flag})
results2.to_csv("results2.csv",index=False)

## LGBM 2

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
lgbm_clf_2=LGBMClassifier(subsample= 0.6, reg_lamda =1, reg_alpha =0.1,
                          n_estimators=100, min_data_in_leaf =10, 
                           max_depth=4, learning_rate =0.05, colsample_bytree = 0.6, alpha =0.7)
lgbm_clf_2.fit(X,y)
lgbm_clf_2.score(X,y)

In [ ]:
predicted_flag=lgbm_clf_2.predict(test)

In [ ]:
results8=pd.DataFrame({"id":test["id"],"buy":predicted_flag})
results8.to_csv("results8.csv",index=False)

## Gradient Booster 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbr_clf=GradientBoostingClassifier(subsample=0.8999999999999999,n_estimators= 1100, 
                                   min_samples_split=12, min_samples_leaf = 4, max_depth =4, 
                                   learning_rate =0.01)
gbr_clf.fit(X,y)

In [ ]:
predicted_flag=gbr_clf.predict(X_test1)

In [ ]:
results8=pd.DataFrame({"id":test["id"],"buy":predicted_flag})
results8.to_csv("results8.csv",index=False)

## CROSS VALIDATION

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=0)

In [ ]:
cross_val_score(lr_clf, X, y, cv=5,scoring='f1')

## Hyper Parameter Tuning

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()

parameters = {'n_estimators': [i for i in range(1000,2000,100)],
              'learning_rate':[i for i in np.arange(0.01,0.05,0.01)],
              'max_depth': [i for i in range(3,10,1)],
              'min_samples_leaf': [i for i in range(1,10,1)],
              'min_samples_split': [i for i in range(5,15,1)],
              'subsample':[i for i in np.arange(0.4,1.0,0.1)]}

randm = RandomizedSearchCV(estimator=model, param_distributions=parameters,
                               cv =5, n_iter = 20, n_jobs=-1,scoring='f1')
randm.fit(X,y)
print(" Results from Random Search" )
print(randm.score(X,y))
print("The best score across ALL searched params:", randm.best_score_)
print("The best parameters across ALL searched params:",randm.best_params_)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Random Search  Best LGBMCLASSIFIER

In [ ]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
parameters = {'n_estimators': [10, 100,200, 500, 1000, 2000],
              'learning_rate':[0.01,0.02,0.05,0.1],
              'max_depth': [1,2,3,4,5,6,7,8],
              'min_data_in_leaf': [1,5,10,20,30],
              'subsample':[0.5,0.6,0.7,0.8,0.9],
              'colsample_bytree':[0.5,0.6,0.7,0.8,0.9],
              'alpha':[0.5,0.6,0.7,0.8,0.9],
              'reg_alpha':[0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
              'reg_lamda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

randm = RandomizedSearchCV(estimator=model, param_distributions=parameters,
                               cv = 5, n_iter = 20, n_jobs=-1,scoring='f1')
randm.fit(X,y)
print(" Results from Random Search " )
#print("The best estimator across ALL searched params:",randm.best_estimator_)
print("Trainning Score",randm.score(X,y))
#print("Testing Score",randm.score(X_test_std,Y_test))
print("The best parameters across ALL searched params:",randm.best_params_)